In [1]:
!pip install textattack transformers sentencepiece pandas tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 74.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 62.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.7/445.7 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
from textattack.augmentation import CLAREAugmenter
from tqdm import tqdm
import random
import os

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
# === PATHS ===
original_path = "/content/drive/MyDrive/merged_question_intent_dataset.csv"  # ✅ Your raw input dataset
checkpoint_path = "/content/drive/MyDrive/augmented_backup.csv"  # ✅ Progress is saved here
final_output_path = "/content/drive/MyDrive/random_augmented_balanced_dataset.csv"  # ✅ Final merged file

# === LOAD DATA ===
original_df = pd.read_csv(original_path)

# Load or create backup file
try:
    augmented_df = pd.read_csv(checkpoint_path)
    print(f"✅ Resuming with {len(augmented_df)} augmented rows already present.")
except FileNotFoundError:
    augmented_df = pd.DataFrame(columns=["text", "intent"])
    print("⚠️ No backup found. Starting fresh.")

# === COUNT CURRENT SAMPLES PER CLASS ===
original_counts = original_df['intent'].value_counts().to_dict()
aug_counts = augmented_df['intent'].value_counts().to_dict()

# Pick target counts
random.seed(42)  # For reproducibility
target_counts = {cls: random.randint(1300, 1800) for cls in original_counts.keys()}

# Compute what's still needed
classes_to_augment = {}
for cls, orig_count in original_counts.items():
    already_aug = aug_counts.get(cls, 0)
    remaining = target_counts[cls] - orig_count - already_aug
    if remaining > 0:
        classes_to_augment[cls] = remaining

if not classes_to_augment:
    print("🎉 All classes are already fully augmented. Nothing to do.")
else:
    print("\n📊 Remaining classes needing augmentation:")
    for cls, count in classes_to_augment.items():
        print(f" → {cls}: {count} more")


✅ Resuming with 422 augmented rows already present.

📊 Remaining classes needing augmentation:
 → ask more: 225 more
 → disease manifestations: 40 more


In [ ]:
# === SETUP AUGMENTER ===
augmenter = CLAREAugmenter(model="roberta-base", transformations_per_example=1)
grouped = original_df.groupby("intent")
augmented_rows = augmented_df.values.tolist()

# === AUGMENT ONLY WHAT'S MISSING ===
for intent, needed in classes_to_augment.items():
    print(f"\n🔧 Augmenting '{intent}' → Need: {needed}")
    group = grouped.get_group(intent)
    texts = group['text'].tolist()
    generated = 0

    with tqdm(total=needed, desc=f"Generating '{intent}'", ncols=100) as pbar:
        while generated < needed:
            for text in texts:
                if generated >= needed:
                    break
                try:
                    aug_texts = augmenter.augment(text)
                    if aug_texts:
                        augmented_rows.append((aug_texts[0], intent))
                        generated += 1
                        pbar.update(1)

                        # Save progress every 50 augmentations
                        if generated % 50 == 0:
                            pd.DataFrame(augmented_rows, columns=["text", "intent"]).to_csv(checkpoint_path, index=False)
                except Exception as e:
                    print(f"⚠️ Error on: {e}")
                    continue


If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`



🔧 Augmenting 'ask more' → Need: 225


Generating 'ask more':  95%|██████████████████████████████████▏ | 214/225 [1:26:15<00:53,  4.82s/it]

In [ ]:
# === SAVE FINAL MERGED DATA ===
aug_df = pd.DataFrame(augmented_rows, columns=["text", "intent"])
final_df = pd.concat([original_df, aug_df]).sample(frac=1).reset_index(drop=True)
final_df.to_csv(final_output_path, index=False)

print(f"\n✅ Done! Final dataset saved to:\n📁 {final_output_path}")
print(f"📊 Total records: {len(final_df)}")
